In [23]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import mode 
from scipy.linalg import det
from matplotlib.pylab import inv

In [24]:
# Limpa o console 
os.system('cls' if os.name == 'nt' else 'clear')
part1 = 'subject0'
part2 = 'subject'
part3 = ['.centerlight', '.glasses', '.happy', '.leftlight', '.noglasses', '.normal', '.rightlight', '.sad', '.sleepy', '.surprised', '.wink']

Nind = 15   # Quantidade de individuos (classes)
Nexp = len(part3)  # Quantidade de expressoes

X = []  # Lista para acumular imagens vetorizadas
Y = []  # Lista para acumular o rotulo (identificador) do individuo

print("Iniciando carregamento e pré-processamento das imagens...")

Iniciando carregamento e pré-processamento das imagens...


In [25]:
for i in range(1, Nind + 1):  # Indice para os individuos
    individuo = i
    # print(f"Processando indivíduo: {individuo}") # Descomente para ver o progresso detalhado
    for j in range(Nexp):   # Indice para expressoes
        if i < 10:
            nome = f"{part1}{i}{part3[j]}"    # Monta o nome do arquivo de imagem
        else:
            nome = f"{part2}{i}{part3[j]}"

        try:
            Img = Image.open(nome).convert('L')  # le imagem e converte para escala de cinza
        except FileNotFoundError:
            print(f"Aviso: Imagem '{nome}' não encontrada. Pulando.")
            continue
        except Exception as e:
            print(f"Erro ao carregar a imagem '{nome}': {e}. Pulando.")
            continue

        # Redimensiona imagem
        Ar = Img.resize((20, 20))

        An = np.array(Ar) # An = Ar.copy() # An=imnoise(Ar,'gaussian',0,0.005);  # adiciona ruido (requer implementação de ruído)

        A = An.astype(np.float64) / 255.0  # converte para double precision (normaliza para 0-1)

        a = A.flatten()  # Etapa de vetorizacao: Empilhamento das colunas

        ROT = i   # Rotulo = indice do individuo

        X.append(a) # Coloca cada imagem vetorizada como linha da matriz X (será transposta depois)
        Y.append(ROT) # Coloca o rotulo de cada vetor

X = np.array(X).T  # Transpõe para ter cada imagem vetorizada como coluna (como no MATLAB)
Y = np.array(Y)

print("Imagens carregadas e vetorizadas. Dimensão de X:", X.shape)
print("Dimensão de Y:", Y.shape)


Imagens carregadas e vetorizadas. Dimensão de X: (400, 165)
Dimensão de Y: (165,)


In [ ]:
q = 25 # Número de componentes principais
pca = PCA(n_components=q)
X_reduced = pca.fit_transform(X.T).T # Aplica PCA nas imagens (linhas de X.T) e transpõe de volta

# Variância explicada acumulada
VEq = np.cumsum(pca.explained_variance_ratio_)
plt.figure()
plt.plot(range(1, len(VEq) + 1), VEq, 'r-', linewidth=3)
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Variância Explicada Acumulada')
plt.title('Variância Explicada Acumulada por PCA')
plt.grid(True)
plt.show()

X = X_reduced # X agora é o X transformado pelo PCA

Z = np.vstack((X, Y))  # Formato 01 vetor de atributos por coluna: DIM(Z) = (p+1)xN
Z = Z.T     # Formato 01 vetor de atributos por linha: DIM(Z) = Nx(p+1)

print("Dados após PCA. Dimensão de X (reduzido):", X.shape)
print("Dimensão de Z (final):", Z.shape)

In [ ]:
np.savetxt('recfaces.dat', Z, fmt='%.6f') # fmt='%.6f' para manter a precisão float

print("\nProcessamento de faces concluído. Dados salvos em 'recfaces.dat'")

In [ ]:

# Funções placeholder para os classificadores 



def quadratico(D, Nr, Ptrain):
    X_data = D[:, :-1]
    y_labels = D[:, -1]

    classes = np.unique(y_labels)
    num_classes = len(classes)

    TX_OK = np.zeros(Nr)
    
    for r in range(Nr):
        # 1. Divisão dos dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(
            X_data, y_labels, test_size=(100 - Ptrain) / 100.0, stratify=y_labels, random_state=r #
        )

        # Dicionários para armazenar centroides e matrizes de covariância
        centroids = {}
        cov_matrices = {}
        inv_cov_matrices = {}
        det_cov_matrices = {}
        # 2. Estimar o vetor centroide e a matriz de covariância de cada classe (Passo 1)
        for c in classes:
            # Selecionar exemplos da classe 'c' no conjunto de treinamento
            X_train_c = X_train[y_train == c]
            Ni = X_train_c.shape[0]

            if Ni == 0:
                print(f"Aviso: Classe {c} não tem exemplos no conjunto de treinamento nesta repetição. Pulando.")
                continue

            # m_i: vetor centroide
            centroids[c] = np.mean(X_train_c, axis=0)

            # C_i: matriz de covariância
            # np.cov espera que as linhas sejam as observações e colunas as características.
            # Se X_train_c já está nesse formato, np.cov(X_train_c) está correto.
            Ci = np.cov(X_train_c, rowvar=False) # rowvar=False -> cada coluna é uma variável, cada linha é uma observação

            # Adicionar uma pequena regularização
            #epsilon = 1e-6 * np.eye(Ci.shape[0])
            #Ci_reg = Ci + epsilon

            # Calcular inversa e determinante (para evitar recalcular dentro do loop de teste)
            try:
                inv_cov_matrices[c] = inv(Ci)
                det_cov_matrices[c] = det(Ci)
            except np.linalg.LinAlgError:
                print(f"Erro: Matriz de covariância para classe {c} é singular ou mal-condicionada. Adicione mais regularização.")
                # Se ocorrer um erro, pode-se pular esta classe ou atribuir um valor alto para o discriminante.
                inv_cov_matrices[c] = None
                det_cov_matrices[c] = float('inf')


        correct_predictions = 0
        # Loop para classificar cada amostra no conjunto de teste
        for i, x_test_sample in enumerate(X_test):
            true_label = y_test[i]
            discriminant_values = {}

            for c in classes:
                if c not in centroids or inv_cov_matrices.get(c) is None:
                    # Se a classe não foi treinada ou houve erro na covariância, atribui um valor alto
                    discriminant_values[c] = float('inf')
                    continue

                m_i = centroids[c]
                inv_Ci = inv_cov_matrices[c]
                det_Ci = det_cov_matrices[c]

                # (x - m_i)
                diff = x_test_sample - m_i
                # 3. Calcular as distâncias quadráticas (Passo 2)
                Qi_x = diff @ inv_Ci @ diff.T

    # Deve retornar STATS, TX_OK, X, m, S, posto
    print("Executando quadratico...")
    # Exemplo de retorno (substitua com a lógica real)
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None, None, None, None

def variante1(D, Nr, Ptrain, param):
    # Implemente a lógica da função 'variante1' aqui
    # Deve retornar STATS, TX_OK, X, m, S, posto
    print("Executando variante1...")
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None, None, None, None

def variante2(D, Nr, Ptrain):
    # Implemente a lógica da função 'variante2' aqui
    # Deve retornar STATS, TX_OK, X, m, S, posto
    print("Executando variante2...")
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None, None, None, None

def variante3(D, Nr, Ptrain, param):
    # Implemente a lógica da função 'variante3' aqui
    # Deve retornar STATS, TX_OK, X, m, S, posto
    print("Executando variante3...")
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None, None, None, None

def variante4(D, Nr, Ptrain):
    # Implemente a lógica da função 'variante4' aqui
    # Deve retornar STATS, TX_OK, X, m, S, posto
    print("Executando variante4...")
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None, None, None, None

def linearMQ(D, Nr, Ptrain):
    # Implemente a lógica da função 'linearMQ' aqui
    # Deve retornar STATS, TX_OK, W
    print("Executando linearMQ...")
    return {'accuracy': np.random.rand() * 100}, np.random.rand(Nr) * 100, None



In [ ]:
# Carrega os dados
try:
    D = np.loadtxt('recfaces.dat')
except FileNotFoundError:
    print("Erro: 'recfaces.dat' não encontrado. Por favor, execute 'face_preprocessing_column.py' primeiro.")
    exit()

Nr = 50  # No. de repeticoes
Ptrain = 80  # Porcentagem de treinamento

# Executa os classificadores
start_time = time.time()
STATS_0, TX_OK0, X0, m0, S0, posto0 = quadratico(D, Nr, Ptrain)
Tempo0 = time.time() - start_time

start_time = time.time()
STATS_1, TX_OK1, X1, m1, S1, posto1 = variante1(D, Nr, Ptrain, 0.01)
Tempo1 = time.time() - start_time

start_time = time.time()
STATS_2, TX_OK2, X2, m2, S2, posto2 = variante2(D, Nr, Ptrain)
Tempo2 = time.time() - start_time

start_time = time.time()
STATS_3, TX_OK3, X3, m3, S3, posto3 = variante3(D, Nr, Ptrain, 0.5)
Tempo3 = time.time() - start_time

start_time = time.time()
STATS_4, TX_OK4, X4, m4, S4, posto4 = variante4(D, Nr, Ptrain)
Tempo4 = time.time() - start_time

start_time = time.time()
STATS_5, TX_OK5, W = linearMQ(D, Nr, Ptrain)
Tempo5 = time.time() - start_time

In [ ]:
# Exibe as estatísticas
print("\nSTATS_0:\n", pd.DataFrame([STATS_0]))
print("\nSTATS_1:\n", pd.DataFrame([STATS_1]))
print("\nSTATS_2:\n", pd.DataFrame([STATS_2]))
print("\nSTATS_3:\n", pd.DataFrame([STATS_3]))
print("\nSTATS_4:\n", pd.DataFrame([STATS_4]))
print("\nSTATS_5:\n", pd.DataFrame([STATS_5]))

TEMPOS = np.array([Tempo0, Tempo1, Tempo2, Tempo3, Tempo4, Tempo5])
print("\nTEMPOS:", TEMPOS)

# Boxplot
data_to_plot = [TX_OK0, TX_OK1, TX_OK2, TX_OK3, TX_OK4, TX_OK5]
labels = ["Quadratico", "Variante 1", "Variante 2", "Variante 3", "Variante 4", "MQ"]

plt.figure(figsize=(10, 6))
plt.boxplot(data_to_plot)
plt.xticks(range(1, len(labels) + 1), labels, rotation=45, ha='right')
plt.title('Conjunto Coluna')
plt.xlabel('Classificador')
plt.ylabel('Taxas de acerto')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()